In [38]:
import pandas as pd
import ast
from pandarallel import pandarallel

In [74]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [27]:
pd.set_option('display.max_columns', None)


In [14]:
df = pd.read_csv('data/merged_data/committer_detailed_info.csv', index_col = 0)

In [15]:
for col in ['actor_id', 'info', 'names', 'emails', 'institutions', 'user types']: 
    df[col] = df[col].apply(lambda x: ast.literal_eval(x))

In [23]:
df['user types'].apply(lambda x: x[0] if len(x) == 1 else 'untraceable' if len(x) == 0 else 'multiple categories').value_counts(normalize = True)

user types
personal                0.443434
untraceable             0.342091
multiple categories     0.121526
company                 0.065408
academic institution    0.027540
Name: proportion, dtype: float64

In [67]:
df_personal = df[df['user types'].apply(lambda x: 'personal' in x)]

In [24]:
pr_data = pd.read_csv('data/merged_data/cleaned_pr_data.csv', index_col = 0)
pr_data.reset_index(drop = True, inplace = True)
for col in ['pr_requested_reviewers','pr_requested_teams','pr_label','pr_actor_id_list','pr_requested_reviewers_list',
            'pr_assignees_list','pr_requested_teams_list','pr_actors','pr_commit_actors','all_pr_actors','commit_actor_id_list']:
    pr_data[col] = pr_data[col].parallel_apply(ast.literal_eval)

/tmp/user/20506/ipykernel_3147177/3136028249.py:1: DtypeWarning: Columns (5,6,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  pr_data = pd.read_csv('data/merged_data/cleaned_pr_data.csv', index_col = 0)


In [152]:
actors = pd.to_numeric(df_personal['actor_id'].explode().tolist())

In [174]:
actor_list = pd.concat([pr_data['all_pr_actors'].explode(),
           pr_data['pr_assignees_list'].explode(),
           pr_data['pr_requested_reviewers_list'].explode(),
           pr_data['pr_merged_by_id'].explode()]).dropna()
actor_list = actor_list[actor_list.apply(lambda x: x != '[]')]
actor_list_dup = pd.to_numeric(actor_list.drop_duplicates())
actor_list_dup_present = actor_list_dup[actor_list_dup.isin(actors)]

In [175]:
actor_list_present = actor_list[pd.to_numeric(actor_list).isin(actor_list_dup_present)]

In [182]:
pr_data_subset = pr_data.loc[actor_list_present.index.drop_duplicates()]

In [194]:
# first day/first pr someone opened as a potential contributor

In [ ]:
pr_data_subset_exp = pr_data_subset.explode(['pr_actor_id_list'])
pr_data_subset_exp = pr_data_subset_exp.explode(['pr_actors'])

In [ ]:
pr_data_subset_exp['created_at'] = pd.to_datetime(pr_data_subset_exp['created_at'])

In [205]:
pr_data_subset_exp

,created_at,repo_id,pr_id,pr_number,pr_locked,pr_merged_at,pr_closed_at,pr_updated_at,pr_commits,pr_additions,pr_deletions,pr_changed_files,pr_assignees,pr_requested_reviewers,pr_requested_teams,pr_merged_by_login,pr_merged_by_id,pr_merged_by_type,pr_merged_by_site_admin,pr_label,retrieved_commits,pr_actor_id_list,repo_name,pr_state,pr_commits_wt,pr_additions_wt,pr_deletions_wt,pr_changed_files_wt,retrieved_commits_wt,pr_assignees_list,pr_requested_reviewers_list,pr_requested_teams_list,closed_wt,merged_wt,pr_actors,pr_commit_actors,all_pr_actors,pr_orgs,pr_commit_orgs,all_pr_orgs,commit_actor_id_list
0,2023-08-29 19:28:10+00:00,61572326,1443376527,6425.0,False,2023-08-29 19:28:09+00:00,2023-08-29 19:28:09+00:00,2023-08-29 19:28:09+00:00,376.0,8.0,14.0,2.0,[],"[155673, 19335594]",[6821073],emmettbutler,723615.0,User,False,[changelog/no-changelog],250,723615 | closed | NAN ORG | CONTRIBUTOR,DataDog/dd-trace-py,closed,1.504,0.032,0.056,0.008,1.0,[],"[155673, 19335594]",[6821073],0.004,1.0,[723615],[723615],[723615],[],[],[],[723615 | NAN ORG | closed]
0,2023-08-29 19:28:10+00:00,61572326,1443376527,6425.0,False,2023-08-29 19:28:09+00:00,2023-08-29 19:28:09+00:00,2023-08-29 19:28:09+00:00,376.0,8.0,14.0,2.0,[],"[155673, 19335594]",[6821073],emmettbutler,723615.0,User,False,[changelog/no-changelog],250,723615 | open | NAN ORG | CONTRIBUTOR,DataDog/dd-trace-py,closed,1.504,0.032,0.056,0.008,1.0,[],"[155673, 19335594]",[6821073],0.004,1.0,[723615],[723615],[723615],[],[],[],[723615 | NAN ORG | closed]
1,2023-08-28 20:51:17+00:00,61572326,1443089371,6418.0,False,2023-08-28 20:51:15+00:00,2023-08-28 20:51:15+00:00,2023-08-28 20:51:16+00:00,340.0,104.0,65.0,4.0,[],"[155673, 19335594, 32471391, 35776586]",[],emmettbutler,723615.0,User,False,[changelog/no-changelog],250,723615 | closed | NAN ORG | CONTRIBUTOR,DataDog/dd-trace-py,closed,1.360,0.416,0.260,0.016,1.0,[],"[155673, 19335594, 32471391, 35776586]",[],0.004,1.0,[723615],[723615],[723615],[],[],[],[723615 | NAN ORG | closed]
2,2023-08-28 13:30:54+00:00,61572326,1439729083,6396.0,False,2023-08-28 13:30:52+00:00,2023-08-28 13:30:52+00:00,2023-08-28 13:30:53+00:00,318.0,71.0,114.0,4.0,[],"[155673, 1466585, 32471391]",[],emmettbutler,723615.0,User,False,[changelog/no-changelog],250,723615 | closed | NAN ORG | CONTRIBUTOR,DataDog/dd-trace-py,closed,1.272,0.284,0.456,0.016,1.0,[],"[155673, 1466585, 32471391]",[],0.004,1.0,[723615],[723615],[723615],[],[],[],[723615 | NAN ORG | closed]
5,2023-08-24 16:27:54+00:00,61572326,1439727861,6395.0,False,2023-08-24 16:27:53+00:00,2023-08-24 16:27:53+00:00,2023-08-24 16:27:53+00:00,297.0,98.0,89.0,6.0,[],"[155673, 6321485, 19335594, 35776586, 62392438]",[],emmettbutler,723615.0,User,False,"[changelog/no-changelog, ASM]",250,723615 | closed | NAN ORG | CONTRIBUTOR,DataDog/dd-trace-py,closed,1.188,0.392,0.356,0.024,1.0,[],"[155673, 6321485, 19335594, 35776586, 62392438]",[],0.004,1.0,[723615],[723615],[723615],[],[],[],[723615 | NAN ORG | closed]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885837,2017-09-07 16:14:26+00:00,42750248,139847597,261.0,False,NaN,NaN,2017-09-07 16:14:25+00:00,1.0,1.0,1.0,1.0,[],"[313157, 1957636]",[],NaN,NaN,NaN,NaN,[],1,10051651 | open | NAN ORG | CONTRIBUTOR,watson-developer-cloud/python-sdk,open,1.000,1.000,1.000,1.000,1.0,[],"[313157, 1957636]",[],0.000,0.0,[10051651],[10051651],[10051651],[],[],[],[10051651 | NAN ORG | open]
885939,2017-08-30 23:20:52+00:00,59868262,138549977,505.0,False,NaN,NaN,2017-08-30 23:20:51+00:00,1.0,48.0,7.0,4.0,[],"[1010634, 4605355]",[],NaN,NaN,NaN,NaN,[],1,368057 | open | NAN ORG | MEMBER,aws/chalice,open,1.000,48.000,7.000,4.000,1.0,[],"[1010634, 4605355]",[],0.000,0.0,[368057],[368057],[368057],[],[],[],[368057 | NAN ORG | open]
885965,2017-08-28 20:57:25+00:00,91356408,138036078,210.0,False,NaN,NaN,2017-08-28 20:57:24+00:00,1.0,10.0,3.0,2.0,[],[1308313],[],NaN,NaN,

In [204]:
pr_data_subset_exp[pr_data_subset_exp['pr_actor_id_list'].apply(lambda x: 'open' in x)].groupby(['repo_id','pr_actor_id_list']).agg(
    {'created_at': 'min'}
)

repo_id
46939       2016-08-20 20:22:41+00:00
59235       2015-04-30 01:54:06+00:00
85418       2015-04-21 15:49:04+00:00
120894      2015-01-14 22:22:02+00:00
138745      2015-01-20 09:00:53+00:00
                       ...           
549162933   2022-10-10 21:26:48+00:00
549610884   2023-04-05 07:28:23+00:00
570117301   2022-11-29 14:02:40+00:00
601048507   2023-02-13 09:44:32+00:00
609273697   2023-06-08 19:38:19+00:00
Name: created_at, Length: 6031, dtype: datetime64[ns, UTC]

In [ ]:
1. Economic/extrinsic motivations
    1. Improve programming skill 
        1. How much have they programmed in general?
        2. How much python have they done? 
    2. Solve task related to primary mission
        1. How much is the individual using the library they’re contributing to in their code? 
    3. Career concerns
        1. Are they contributing to a repository owned by a company?
        2. Are they contributing to a trendy project?
        3. Is there project widely downloaded?
        4. Are they contributing to projects with less restrictive licenses?
2. Enjoyment-related motivation
    1. Enjoy OSS contribution
        1. Are they an extensive OSS contributor? Note the overlap
    2. intellectual stimulation/ego gratification: solving hard problems
        1. Issues that are classified as hard? Not good first issue, maybe issues that have tag words associated with “hard”, “difficult”, etc
    3. ego gratification: being significant contributor
        1. projects with less number of contributors, but is still widely used?
3. Community-related motivations
    1. Identification as OSS developer
        1. Are they an extensive OSS contributor
        2. Are they owner/on leadership of other OSS projects? 
    2. Belief that code should be open
        1. What are the licenses of their own repositories/the work they contribute to?
    3. Supporting OSS community
        1. Working on documentation, answering issues in past work
    4. Social Connections
        1. Do they follow people who work on OSS




